This notebook visualizes AIS voyages where there are unusually long delays between subsequent pings (ping deltas) as datashaded segments.

In [ ]:
import warnings
import pandas as pd
import numpy as np
import panel as pn
import datetime as dt
import holoviews as hv
from holoviews.operation.datashader import rasterize, dynspread
from colorcet import fire

hv.extension('bokeh')

In [ ]:
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [ ]:
zone_num=10
basedir = './data/Cleaned_AIS/Zone10_201_01/'
broadcast = pd.read_csv(basedir+'Broadcast.csv', parse_dates=[1])
broadcast.head()

### Helper functions to generate segments between unusually delayed subsequent AIS pings

In [ ]:
def close_to_border(slon, elon, zone, border):
    l,r = (-180+6*(zone-1),-180+6*zone)
    lborder, rborder = l+border, r-border
    if (slon < lborder) or (elon < lborder):
        return True
    if (slon > rborder) or (elon > rborder):
        return True
    return False

def voyage_outlier_coords(voyage_dataframe, deviations, zone, border):
    """
    Output eastings and northing 4-tuples [e1, n1, e2, n2] corresponding to 
    pings 1 and 2 with a temporal delta greater than the requested
    standard deviations than the mean delta.
    """
    coords = []
    voyage_df = voyage_dataframe.sort_index()
    seconds = np.diff(voyage_df.index).astype(int) / (1.0e9)
    delta = (seconds - seconds.mean()) # Deviation from the mean
    outliers = np.abs(delta) > (seconds.std() * deviations)
    outlier_diff_inds = np.nonzero(outliers)[0]
    if len(outlier_diff_inds) == 0:
        return []
    for diff_ind in outlier_diff_inds:
        start = voyage_df.iloc[diff_ind]
        end = voyage_df.iloc[diff_ind+1]
        slat, slon = start['lat'], start['lon']
        elat, elon = end['lat'], end['lon']
        if close_to_border(slon, elon, zone, border):
            continue
        seasting, snorthing = hv.util.transform.lon_lat_to_easting_northing(slon, slat)
        eeasting, enorthing = hv.util.transform.lon_lat_to_easting_northing(elon,elat)
        coords.append((seasting, snorthing, eeasting, enorthing))
    return coords

def voyage_dfs(vessel_df):
    "Given a vessel DataFrame with a single mssid split by voyage ids"
    voyages = list(vessel_df['voyage_id'].unique())
    return {vid: vessel_df[vessel_df['voyage_id']==vid] for vid in voyages}

def outlier_segments(vessels, zone, deviations=5, border=1):
    "For all vessel voyages compute segments for ping pairs exceeding deviation"
    coords = []
    for vessel, vessel_df in vessels.items():
        voyages = voyage_dfs(vessel_df)
        for voyage, voyage_df in voyages.items():
            coords += voyage_outlier_coords(voyage_df, deviations=deviations, zone=zone, border=border)
    return hv.Segments(np.array(coords))

### Split AIS data by mmsi_id into DataFrames with sorted time index

In [ ]:
vessels = {name:df.drop_duplicates().sort_values(by='date_time').set_index('date_time') 
           for name,df in broadcast.groupby('mmsi_id')}

   ### Projected AIS into WebMercator

In [ ]:
%%time
eastings, northings = hv.util.transform.lon_lat_to_easting_northing(broadcast['lon'], broadcast['lat'])

### Datashade all AIS data as points and unusually delayed pings as segments

Note that when zooming in around Portland, you can see segments crossing land. These mark vessels that generated an AIS ping while out to sea and did not generate another AIS ping until inland.

In [ ]:
deviations=20 # Segments show ping deltas more than 20 standard deviations from the mean
border=0.5 # Border from UTM boundary in degrees of longitude to filter by

In [ ]:
def zone(i):
    """
    Return plottable bounds object for a given UTM zone
    (see https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system#UTM_zone)
    """
    lrbt = ((-180+6*(i-1),-180+6*i),(-80,84))
    m    = hv.util.transform.lon_lat_to_easting_northing(*lrbt)
    bnds = hv.Bounds((m[0][0],m[1][0],m[0][1],m[1][1])).opts(color="white") 
    text = hv.Text(m[0][0]+(m[0][1]-m[0][0])/2, 0, f"{i}").opts(color="white", text_font_size="5pt")
    return bnds * text

In [ ]:
tiles = hv.element.tiles.ESRI().redim(x='easting', y='northing')
points = rasterize(hv.Points(pd.DataFrame({'northing':northings, 
                                           'easting':eastings}), ['easting', 'northing']))
(tiles * points.opts(cmap=fire[180:], width=700, height=400, cnorm='eq_hist', alpha=0.5) * zone(zone_num)
 * dynspread(rasterize(outlier_segments(vessels, zone_num, deviations=deviations, border=border))).opts(cmap='Reds_r'))